In [1]:
# boilerplate
import random
import tensorflow as tf
sess = tf.InteractiveSession()
import tensorflow_fold as td
import gensim
import numpy as np
import math

In [2]:
UNK = 'UNK'
# This file contains the dataset in a useful way. We populate a list of
# Trees to train/test our Neural Nets such that each Tree contains any
# number of Node objects.

# The best way to get a feel for how these objects are used in the program is to drop pdb.set_trace() in a few places throughout the codebase
# to see how the trees are used.. look where loadtrees() is called etc..

class Node:  # a node in the tree
    def __init__(self, label = None, word=None):
        self.label = label
        self.word = word
        self.parent = None  # reference to parent
        self.left = None  # reference to left child
        self.right = None  # reference to right child
        # true if I am a leaf (could have probably derived this from if I have
        # a word)
        self.isLeaf = False
        # true if we have finished performing fowardprop on this node (note,
        # there are many ways to implement the recursion.. some might not
        # require this flag)
        self.level = 0
        #defeault intitialziation of depth
        self.has_label = False
           
class Tree:

    def __init__(self, treeString, openChar='(', closeChar=')', label_size = 18):
        tokens = []
        self.open = '('
        self.close = ')'
        for toks in treeString.strip().split():
            tokens += list(toks)
        self.root = self.parse(tokens, label_size = label_size)
          
        self.self_binarize()
        self.depth = get_depth(self.root)
        self.levels = max(math.floor(math.log(float(self.depth)) / math.log(float(2)))-1, 0)
        self.binary = check_for_binarization(self.root)
        #propagate_label(self.root, self.levels, self.depth)
        self.labels = get_labels(self.root)
        self.labelcount = count_labels(self.root)

    def parse(self, tokens, parent=None, label_size = 18):
        
        assert tokens[0] == self.open, "Malformed tree"
        assert tokens[-1] == self.close, "Malformed tree"
        
        split = 1  # position after open 
        marker  = 1
        countOpen = countClose = 0
        label = None
        if (split + label_size) < len(tokens):
         str1 = ''.join(tokens[split: (split + label_size)])
         if str1.isdigit():
        
            label = tokens[split: (split + label_size)]
            label = np.asarray(label).astype(int)
            split += label_size
            marker += label_size 
                
        if tokens[split] == self.open:
            countOpen += 1
            split += 1
        # Find where left child and right child split
        while countOpen != countClose:
            if tokens[split] == self.open:
                countOpen += 1
            if tokens[split] == self.close:
                countClose += 1
            split += 1

        # New node
        if isinstance(label, np.ndarray):
         node = Node(label)  
         node.has_label = True
        else:
         node = Node()   
        
        if parent: 
         node.parent = parent
         node.level = parent.level + 1

        # leaf Node
        if countOpen == 0:
            node.word = ''.join(tokens[marker:-1])  # distinguish between lower and upper. Important for words like Apple
            node.isLeaf = True
            return node

        node.left = self.parse(tokens[marker:split], parent=node)
        if  (tokens[split] == self.open) :
         node.right = self.parse(tokens[split:-1], parent=node)

        return node
     

    def get_words(self):
        leaves = getLeaves(self.root)
        words = [node.word for node in leaves]
        return words

    def self_binarize(self):
     
     def binarize_tree(node):
      
      if node.isLeaf:
       return
      elif ((node.left is not None) & (node.right is not None)):
       binarize_tree(node.left)
       binarize_tree(node.right)
      else:
       #fuse parent node with child node
       node.left.label = node.label
       node.left.level -= 1
       
       if (node.level != 0):
        if (node.parent.right is node):
          node.parent.right = node.left
        else:
          node.parent.left = node.left 
        node.left.parent = node.parent
       
       else:
        self.root = node.left
        node.left.parent = None
        self.root.has_label = True
       
       binarize_tree(node.left)
     binarize_tree(self.root)

def check_for_binarization(node):
      
      if node.isLeaf:
        return True
      elif (node.right is None):
        return False 
      else:
       b1 = check_for_binarization(node.left) 
       b2 = check_for_binarization(node.right)
      return (b1 & b2)
         
def count_labels(node):
    if node is None:
     return 0
    if node.has_label == False:
     return 0
    count = 0
    if node.has_label == True:
      count = 1
      return count + count_labels(node.left) + count_labels(node.right)
    
 
def binarize(node):
     
      if node.isLeaf:
       return
      elif ((node.left is not None) & (node.right is not None)):
       binarize(node.left)
       binarize(node.right)
      else:
       #fuse parent node with child node
       node.left.label = node.label
       node.left.level -= 1
       if (node.parent.right is node):
        node.parent.right = node.left
       else:
        node.parent.left = node.left
       node.left.parent = node.parent
       binarize(node.left)



def get_depth(node):
    if node is None:
         return

    if node.isLeaf:
      return 0
    return (1+ max(get_depth(node.left), get_depth(node.right)))  
        
def propagate_label(node, levels, depth):
    
    if node is None:
         return
    if (node.level > levels):
    #if (node.level > levels) or ((get_depth(node) + node.level) < depth): 
         return
    
    if node.parent:
     node.label = node.parent.label
     node.has_label = True
    propagate_label(node.left, levels, depth)
    propagate_label(node.right, levels, depth)

   

def leftTraverse(node, nodeFn=None, args=None):
    """
    Recursive function traverses tree
    from left to right. 
    Calls nodeFn at each node
    """
    if node is None:
        return
    leftTraverse(node.left, nodeFn, args)
    leftTraverse(node.right, nodeFn, args)
    nodeFn(node, args)


def getLeaves(node):
    if node is None:
        return []
    if node.isLeaf:
        return [node]
    else:
        return getLeaves(node.left) + getLeaves(node.right)


def get_labels(node):
    if node is None:
        return []
    if node.has_label == False:
        return []
    return get_labels(node.left) + get_labels(node.right) + [node.label]



def clearFprop(node, words):
    node.fprop = False


def loadTrees(dataSet='train'):
    """
    Loads training trees. Maps leaf node words to word ids.
    """
    file = 'trees/%s.txt' % dataSet
    print ("Loading %s trees.." % dataSet)
    with open(file, 'r') as fid:
        
        trees = [Tree(l) for l in fid.readlines()]

    return trees


In [3]:
def loadmodel():
    print("Loading Google Word2vecs....")
    model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary = True)
    return model


In [4]:
train_trees = loadTrees('train')
dev_trees = loadTrees('dev')
test_trees = loadTrees('test')

Loading train trees..
Loading dev trees..
Loading test trees..


In [5]:
train_nodes = [t.root for t in train_trees]
dev_nodes = [t.root for t in dev_trees]
test_nodes = [t.root for t in test_trees]

In [6]:
model = loadmodel()

Loading Google Word2vecs....


In [7]:
def filter_model(model):
    filtered_dict = {}
    trees = loadTrees('train') + loadTrees('dev') + loadTrees('test')
    words = [t.get_words() for t in trees]
    vocab = set()
    for word in words:
        vocab.update(word)
    for word in vocab:
        if word in model.vocab:
            filtered_dict[word] = model[word]
    return filtered_dict

In [8]:
filtered_model = filter_model(model)

Loading train trees..
Loading dev trees..
Loading test trees..


In [9]:
def load_embeddings(filtered_model):
  """Loads embedings, returns weight matrix and dict from words to indices."""
  print('loading word embeddings')
  weight_vectors = []
  word_idx = {}
  for word, vector in filtered_model.items():
    word_idx[word] = len(weight_vectors)
    weight_vectors.append(np.array(vector, dtype=np.float32))
  # Random embedding vector for unknown words.
  weight_vectors.append(np.random.uniform(
      -0.05, 0.05, weight_vectors[0].shape).astype(np.float32))
  return np.stack(weight_vectors), word_idx

In [10]:
weight_matrix, word_idx = load_embeddings(filtered_model)

loading word embeddings


In [11]:
class BinaryTreeLSTMCell(tf.contrib.rnn.BasicLSTMCell):
  """LSTM with two state inputs.

  This is the model described in section 3.2 of 'Improved Semantic
  Representations From Tree-Structured Long Short-Term Memory
  Networks' <http://arxiv.org/pdf/1503.00075.pdf>, with recurrent
  dropout as described in 'Recurrent Dropout without Memory Loss'
  <http://arxiv.org/pdf/1603.05118.pdf>.
  """

  def __init__(self, num_units, keep_prob=1.0):
    """Initialize the cell.

    Args:
      num_units: int, The number of units in the LSTM cell.
      keep_prob: Keep probability for recurrent dropout.
    """
    super(BinaryTreeLSTMCell, self).__init__(num_units)
    self._keep_prob = keep_prob

  def __call__(self, inputs, state, scope=None):
    with tf.variable_scope(scope or type(self).__name__):
      lhs, rhs = state
      c0, h0 = lhs
      c1, h1 = rhs
      concat = tf.contrib.layers.linear(
          tf.concat([inputs, h0, h1], 1), 5 * self._num_units)

      # i = input_gate, j = new_input, f = forget_gate, o = output_gate
      i, j, f0, f1, o = tf.split(value=concat, num_or_size_splits=5, axis=1)

      j = self._activation(j)
      if not isinstance(self._keep_prob, float) or self._keep_prob < 1:
        j = tf.nn.dropout(j, self._keep_prob)

      new_c = (c0 * tf.sigmoid(f0 + self._forget_bias) +
               c1 * tf.sigmoid(f1 + self._forget_bias) +
               tf.sigmoid(i) * j)
      new_h = self._activation(new_c) * tf.sigmoid(o)

      new_state = tf.contrib.rnn.LSTMStateTuple(new_c, new_h)

      return new_h, new_state

In [12]:
keep_prob_ph = tf.placeholder_with_default(1.0, [])

In [13]:
lstm_num_units = 300  # Tai et al. used 150, but our regularization strategy is more effective
tree_lstm = td.ScopedLayer(
      tf.contrib.rnn.DropoutWrapper(
          BinaryTreeLSTMCell(lstm_num_units, keep_prob=keep_prob_ph),
          input_keep_prob=keep_prob_ph, output_keep_prob=keep_prob_ph),
      name_or_scope='tree_lstm')

In [14]:
NUM_ASPECTS = 18  # number of aspects
NUM_POLARITY = 4 #number of polarity classes assicated with an aspect (1 = mildly +ve or -ve, 2 = negative, 3 = positive)
output_layer = td.FC(NUM_ASPECTS*NUM_POLARITY, activation=None, input_keep_prob=0.9,  name='output_layer')

In [15]:
word_embedding = td.Embedding(
    *weight_matrix.shape, initializer=weight_matrix, name='word_embedding', trainable = False)

In [16]:
embed_subtree = td.ForwardDeclaration(name='embed_subtree')

In [17]:
def logits_and_state():
  """Creates a block that goes from tokens to (logits, state) tuples."""
  unknown_idx = len(word_idx)
  lookup_word = lambda word: word_idx.get(word, unknown_idx)
  
  word2vec = (td.GetItem(0) >> td.InputTransform(lookup_word) >>
              td.Scalar('int32') >> word_embedding)

  pair2vec = (embed_subtree(), embed_subtree())

  # Trees are binary, so the tree layer takes two states as its input_state.
  zero_state = td.Zeros((tree_lstm.state_size,) * 2)
  # Input is a word vector.
  zero_inp = td.Zeros(word_embedding.output_type.shape[0])

  word_case = td.AllOf(word2vec, zero_state)
  pair_case = td.AllOf(zero_inp, pair2vec)

  tree2vec = td.OneOf(len, [(1, word_case), (2, pair_case)])

  return tree2vec >> tree_lstm >> (output_layer, td.Identity())

In [18]:
def tf_node_loss(logits, labels):
  #mask = (labels > 0)
  #actuals = tf.cast(mask, tf.int32) #binarize the labels to compute loss for aspect detection 
  logits2 = tf.reshape(logits, [-1, NUM_ASPECTS,  NUM_POLARITY])
 
  #logits3 = tf.slice(logits2, [0,0,0], [-1,-1, 2])
  #logits4 = tf.slice(logits2, [0,0,1], [-1,-1, -1])
  #labels2 = tf.boolean_mask(labels, mask) -1
  #logits5 = tf.boolean_mask(logits4, mask )
  #loss1 = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits5, labels=labels2))
  loss2 = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits2, labels=labels), axis = 1)
  #loss3 = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits4, labels=labels), axis = 1)
  return loss2

In [19]:
def tf_tpr(logits, labels):
  logits2 = tf.nn.softmax(tf.reshape(logits, [-1, NUM_ASPECTS, NUM_POLARITY]))
  predictions2 = ( logits2[:,:, 1] ) > (logits2[:,:, 0])
 
  predictions3 = tf.cast(predictions2, tf.float64)
 
  actuals1 = labels > 0
  actuals2 = tf.cast(actuals1, tf.float64)

  ones_like_actuals = tf.ones_like(actuals2)
  zeros_like_actuals = tf.zeros_like(actuals2)
  ones_like_predictions = tf.ones_like(predictions3)
  zeros_like_predictions = tf.zeros_like(predictions3)

  ans = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
           tf.equal(actuals2, ones_like_actuals), 
           tf.equal(predictions3, ones_like_predictions)
      ), 
      tf.float64
    ), axis = 1
  )

  return ans

In [20]:
def tf_tnr(logits, labels):
  logits2 = tf.nn.softmax(tf.reshape(logits, [-1, NUM_ASPECTS,  NUM_POLARITY]))
  predictions2 = ( logits2[:,:, 1] ) > (logits2[:,:, 0])
  #predictions =tf.cast(tf.argmax(logits2, 2), tf.int32)
  #predictions2 = predictions > 0
  predictions3 = tf.cast(predictions2, tf.float64)
  #actuals =  tf.reshape(labels, [-1, 1])
  actuals1 = labels > 0
  actuals2 = tf.cast(actuals1, tf.float64)

  ones_like_actuals = tf.ones_like(actuals2)
  zeros_like_actuals = tf.zeros_like(actuals2)
  ones_like_predictions = tf.ones_like(predictions3)
  zeros_like_predictions = tf.zeros_like(predictions3)

  ans = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
           tf.equal(actuals2, zeros_like_actuals), 
           tf.equal(predictions3, zeros_like_predictions)
      ), 
      tf.float64
    ), axis = 1
  )

  return ans

In [21]:
def tf_fpr(logits, labels):
  logits2 = tf.nn.softmax(tf.reshape(logits, [-1, NUM_ASPECTS,  NUM_POLARITY]))
  predictions2 = ( logits2[:,:, 1] ) > (logits2[:,:, 0])
  #predictions =tf.cast(tf.argmax(logits2, 2), tf.int32)
  #predictions2 = predictions > 0
  predictions3 = tf.cast(predictions2, tf.float64)
  #actuals =  tf.reshape(labels, [-1, 1])
  actuals1 = labels > 0
  actuals2 = tf.cast(actuals1, tf.float64)

  ones_like_actuals = tf.ones_like(actuals2)
  zeros_like_actuals = tf.zeros_like(actuals2)
  ones_like_predictions = tf.ones_like(predictions3)
  zeros_like_predictions = tf.zeros_like(predictions3)

  ans = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
           tf.equal(actuals2, zeros_like_actuals), 
           tf.equal(predictions3, ones_like_predictions)
      ), 
      tf.float64
    ), axis = 1
  )

  return ans

In [22]:
def tf_fnr(logits, labels):
  logits2 = tf.nn.softmax(tf.reshape(logits, [-1, NUM_ASPECTS,  NUM_POLARITY]))
  predictions2 = ( logits2[:,:, 1] ) > (logits2[:,:, 0])
  predictions3 = tf.cast(predictions2, tf.float64)
  actuals1 = labels > 0
  actuals2 = tf.cast(actuals1, tf.float64)

  ones_like_actuals = tf.ones_like(actuals2)
  zeros_like_actuals = tf.zeros_like(actuals2)
  ones_like_predictions = tf.ones_like(predictions3)
  zeros_like_predictions = tf.zeros_like(predictions3)

  ans = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
           tf.equal(actuals2, ones_like_actuals), 
           tf.equal(predictions3, zeros_like_predictions)
      ), 
      tf.float64
    ), axis = 1
  )

  return ans

In [23]:
def add_metrics(is_root, is_neutral):
  """A block that adds metrics for loss and hits; output is the LSTM state."""
  c = td.Composition(
      name='predict(is_root=%s, is_neutral=%s)' % (is_root, is_neutral))
  with c.scope():
    # destructure the input; (labels, neutral, (logits, state))
    labels = c.input[0]
    logits = td.GetItem(0).reads(c.input[2])
    state = td.GetItem(1).reads(c.input[2])

    loss = td.Function(tf_node_loss)
    td.Metric('all_loss').reads(loss.reads(logits, labels))
    if is_root: td.Metric('root_loss').reads(loss)
   
    tpr = td.Function(tf_tpr)
    tnr = td.Function(tf_tnr)
    fpr = td.Function(tf_fpr)
    fnr = td.Function(tf_fnr)
    td.Metric('all_tpr').reads(tpr.reads(logits, labels))
    td.Metric('all_tnr').reads(tnr.reads(logits, labels))
    td.Metric('all_fpr').reads(fpr.reads(logits, labels))
    td.Metric('all_fnr').reads(fnr.reads(logits, labels))                           
    if is_root: 
        td.Metric('tpr').reads(tpr)
        td.Metric('tnr').reads(tnr)
        td.Metric('fpr').reads(fpr)
        td.Metric('fnr').reads(fnr)
   
    # output the state, which will be read by our by parent's LSTM cell
    c.output.reads(state)
  return c

In [24]:
def tokenize(node):
  group = []
  neutral = '2'
  if node.has_label:
  
    label = node.label
    
    neutral = '1'
  else:
    label = np.zeros((NUM_ASPECTS,),dtype=np.int)
    
  if node.isLeaf:
    group = [node.word]
  else:
    group = [node.left, node.right]
  return label, neutral, group

In [25]:
node = train_nodes[0]
label, neutral, group = tokenize(node)
print (len(group))
print (label.shape)

2
(18,)


In [26]:
def embed_tree(logits_and_state, is_root):
  """Creates a block that embeds trees; output is tree LSTM state."""
  return td.InputTransform(tokenize) >> td.OneOf(
      key_fn=lambda pair: pair[1] == '2',  # label 2 means neutral
      case_blocks=(add_metrics(is_root, is_neutral=False),
                   add_metrics(is_root, is_neutral=True)),
      pre_block=(td.Vector(NUM_ASPECTS, dtype = 'int32'), td.Scalar('int32'), logits_and_state))

In [27]:
model = embed_tree(logits_and_state(), is_root=True)

In [28]:
embed_subtree.resolve_to(embed_tree(logits_and_state(), is_root=False))

In [29]:
compiler = td.Compiler.create(model)
print('input type: %s' % model.input_type)
print('output type: %s' % model.output_type)

input type: PyObjectType()
output type: TupleType(TensorType((300,), 'float32'), TensorType((300,), 'float32'))


In [30]:
metrics = {k: tf.reduce_mean(v) for k, v in compiler.metric_tensors.items()}

In [31]:
LEARNING_RATE = 0.05
KEEP_PROB = 0.6
BATCH_SIZE = 25
EPOCHS = 100
EMBEDDING_LEARNING_RATE_FACTOR = 0.1

In [32]:
train_feed_dict = {keep_prob_ph: KEEP_PROB}
loss = tf.reduce_mean(compiler.metric_tensors['root_loss'])
opt = tf.train.AdagradOptimizer(LEARNING_RATE)
train = opt.minimize(loss)
saver = tf.train.Saver()

/home/impadmin/anaconda2/envs/LSTM/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [33]:
sess.run(tf.global_variables_initializer())

In [34]:
def train_step(batch):
  train_feed_dict[compiler.loom_input_tensor] = batch
  _, batch_loss = sess.run([train, loss], train_feed_dict)
  return batch_loss

In [35]:
def train_epoch(train_set):
  list = [train_step(batch) for batch in td.group_by_batches(train_set, BATCH_SIZE)]
  return sum(list)/ max(len(list), 1)

In [36]:
train_set = compiler.build_loom_inputs(train_nodes)

In [37]:
dev_feed_dict = compiler.build_feed_dict(dev_nodes)

In [38]:
def dev_eval(epoch, train_loss):
  dev_metrics = sess.run(metrics, dev_feed_dict)
  dev_loss = dev_metrics['root_loss']

  tp = dev_metrics['tpr']
  tn = dev_metrics['tnr']
  fp = dev_metrics['fpr']
  fn = dev_metrics['fnr']
  
  tpr = float(tp)/(float(tp) + float(fn))
  fpr = float(fp)/(float(tp) + float(fn))

  

  recall = tpr
  if (float(tp) + float(fp)) > 0:
   precision = float(tp)/(float(tp) + float(fp))
  else: precision = 0.
  if precision + recall > 0:
   f1_score = (2 * (precision * recall)) / (precision + recall)
  else: f1_score = 0.
 
 
  print('epoch:%4d, train_loss: %.3e, dev_loss: %.3e,Task1 Precision: %.3e, Task1 Recall: %.3e, Task1 F1 score: %.3e'
        % (epoch, train_loss, dev_loss, precision, recall, f1_score))
  return f1_score

In [39]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()n locals() and session is not None:
    print('Close interactive session')
    session.close()

SyntaxError: invalid syntax (<ipython-input-39-9f5dc04c7c68>, line 3)

In [39]:
best_accuracy = 0.0
save_path = 'weights/sentiment_model'
for epoch, shuffled in enumerate(td.epochs(train_set, EPOCHS), 1):
  train_loss = train_epoch(shuffled)
  f1_score = dev_eval(epoch, train_loss)
 

InternalError: Blas SGEMM launch failed : a.shape=(340, 900), b.shape=(900, 1500), m=340, n=1500, k=900
	 [[Node: while/Function_35/tree_lstm/fully_connected/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](while/Function_35/tree_lstm/concat, while/Function_17/tree_lstm/fully_connected/MatMul/Enter)]]
	 [[Node: while/Function_6/SparseSoftmaxCrossEntropyWithLogits/Shape_1/_109 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2569_while/Function_6/SparseSoftmaxCrossEntropyWithLogits/Shape_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](^_cloopwhile/Function_6/arg_1/Gather/_11)]]

Caused by op 'while/Function_35/tree_lstm/fully_connected/MatMul', defined at:
  File "/home/impadmin/anaconda2/envs/LSTM/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__